In [3]:
(require racket/pretty)

In [77]:
(require racket/list)

In [6]:
(require racket/match)

In [632]:
(require json)

In [348]:
(require racket/math)

In [359]:
(require racket/format)

In [509]:
(require racket/string)

# Functions

In [658]:
(define (parse-kicad-pcb fname)
    (let ([sexp (call-with-input-file fname
            (lambda (in)
                (read in)))])
        sexp))

In [659]:
(define (kicad->area sexp)
  (match sexp 
       [`(kicad_pcb 
          ,(not `(general ,_ ...)) ...
          (general 
           ,_ ...
           (area ,x1 ,y1 ,x2 ,y2)
           ,_ ...)
          ,_ ...)
         (list x1 y1 x2 y2)]))

In [681]:
;; module has shape and pads, pad belong to 0 or 1 netlist
(define (module-clause->module module-clause)
  (match module-clause
       [`(module ,name
                 ,_ ...
                 (layer ,layer)
                 ,_ ...
                 ;; FIXME z?
                 (at ,x ,y ,z ...)
                 ,(not `(pad ,_ ...)) ...
                 (pad ,_
                      ,type
                      ,shape
                      ;; FIXME padz
                      (at ,padx ,pady ,padz ...)
                      ,_ ...
                      (layers ,layers ...)
                      ;; there might be a net OR NOT
                      (net ,netId ,_) ...
                      )
                 ;; FIXME some modules might not have pads
                 ...
                 ,_ ...
                 )
         ;; module
         (hash 'x x
               'y y
               'pads (for/list ([t type]
                               [s shape]
                               [x padx]
                               [y pady]
                               [net netId]
                                [l layers])
                               (hash 'type (symbol->string t)
                                     'shape (symbol->string s)
                                     'x x
                                     'y y
                                     'net (if (empty? net) null (first net))
                                     'layers (begin 
                                                (let ([res '()])
                                                  (when (or (member 'F.Cu l)
                                                            (member '*.Cu l))
                                                        (set! res (append res '("F"))))
                                                      (when (or (member 'B.Cu l)
                                                                (member '*.Cu l))
                                                            (set! res (append res '("B"))))
                                                      res)

                                                      ))
                              ))
         ]))

In [682]:
(define (kicad->modules sexp)
  (define clauses (match sexp
       [`(kicad_pcb
          ,clauses ...)
         clauses]))
  (define module-clauses (filter (lambda (x)
              (match x
                     [`(module ,_ ...) #t]
                     [else #f]))
        clauses))
  (define modules
      (for/list ([c module-clauses]
          [i (in-naturals)])
         (module-clause->module c)))
  modules
  )

In [683]:
(define (kicad-file->json in-fname out-fname)
  (define sexp (parse-kicad-pcb in-fname))
  ;; clauses
  (define clauses (match sexp
       [`(kicad_pcb
          ,clauses ...)
         clauses]))
  ;; FIXME there might not be "area" clause
  ;; TODO I could use the min and max of all modules
  (define area (kicad->area sexp))
  (define modules (kicad->modules sexp))
  (call-with-output-file out-fname
                       #:exists 'replace
                     (lambda (out)
                       (write-json (hash 'modules modules
                                         'area area)
                                   out))))

In [663]:
(kicad-file->json "./benchmarks/cantact.kicad_pcb" "out.json")

In [684]:
(kicad-file->json "benchmarks/sensorboard/hardware/sensorboard/sensorboard.kicad_pcb" "out.json")

In [700]:
(kicad-file->json "benchmarks/mppt-2420-lc/MPPT_charger_20A.kicad_pcb" "out.json")

match: no matching clause for '(kicad_pcb (version 20171130) (host pcbnew "(5.0.0-rc2-dev-444-g2974a2c10)") (general (thickness 1.6) (drawings 84) (tracks 2076) (zones 0) (modules 149) (nets 104)) (page A4) (title_block (title "MPPT Charger 20A") (date 2017-02-16) (rev 0.9) (company...
  location...:
   eval:2:2
  context...:
   /usr/share/racket/collects/racket/match/gen-match.rkt:88:24: body of top-level
   eval:1:0: kicad-file->json


# DEBUG

In [695]:
(define in-fname "benchmarks/mppt-2420-lc/MPPT_charger_20A.kicad_pcb")

In [696]:
(define sexp (parse-kicad-pcb in-fname))

In [704]:
(display (pretty-format sexp))

'(kicad_pcb
  (version 20171130)
  (host pcbnew "(5.0.0-rc2-dev-444-g2974a2c10)")
  (general
   (thickness 1.6)
   (drawings 84)
   (tracks 2076)
   (zones 0)
   (modules 149)
   (nets 104))
  (page A4)
  (title_block
   (title "MPPT Charger 20A")
   (date 2017-02-16)
   (rev 0.9)
   (company "Libre Solar")
   (comment 1 "Design: Martin Jäger")
   (comment 2 "Website: http://libre.solar"))
  (layers
   (0 Top signal)
   (31 Bottom signal)
   (32 B.Adhes user hide)
   (33 F.Adhes user hide)
   (34 B.Paste user hide)
   (35 F.Paste user hide)
   (36 B.SilkS user hide)
   (37 F.SilkS user)
   (38 B.Mask user)
   (39 F.Mask user hide)
   (40 Dwgs.User user hide)
   (41 Cmts.User user hide)
   (42 Eco1.User user hide)
   (43 Eco2.User user hide)
   (44 Edge.Cuts user)
   (45 Margin user hide)
   (46 B.CrtYd user hide)
   (47 F.CrtYd user)
   (48 B.Fab user hide)
   (49 F.Fab user hide))
  (setup
   (last_trace_width 0.25)
   (user_trace_width 0.25)
   (user_trace_width 0.3)
   (user_trace_w

In [697]:
;; clauses
(define clauses (match sexp
       [`(kicad_pcb
          ,clauses ...)
         clauses]))
  

In [701]:
(define area (kicad->area sexp))

match: no matching clause for '(kicad_pcb (version 20171130) (host pcbnew "(5.0.0-rc2-dev-444-g2974a2c10)") (general (thickness 1.6) (drawings 84) (tracks 2076) (zones 0) (modules 149) (nets 104)) (page A4) (title_block (title "MPPT Charger 20A") (date 2017-02-16) (rev 0.9) (company...
  location...:
   eval:2:2
  context...:
   /usr/share/racket/collects/racket/match/gen-match.rkt:88:24: body of top-level


In [698]:
(define modules (kicad->modules sexp))

In [699]:
modules

'(#hash((pads . (#hash((layers . ("F" . #0=("B"))) (net . 102) (shape . "oval") (type . "thru_hole") (x . 0) (y . 2.54)) #hash((layers . ("F" . #0#)) (net . 27) (shape . "rect") (type . "thru_hole") (x . 0) (y . 0)))) (x . 199.5) (y . 83.5)) #hash((pads . ()) (x . 165.8) (y . 104.8)) #hash((pads . ()) (x . 124.8) (y . 84.2)) #hash((pads . ()) (x . 205.6) (y . 89.9)) #hash((pads . (#hash((layers . ("F" . #0#)) (net . 1) (shape . "circle") (type . "thru_hole") (x . 5) (y . 0)) #hash((layers . ("F" . #0#)) (net . 93) (shape . "rect") (type . "thru_hole") (x . 0) (y . 0)))) (x . 137.5) (y . 114)) #hash((pads . (#hash((layers . ("F" . #0#)) (net . 1) (shape . "circle") (type . "thru_hole") (x . 7.5) (y . 0)) #hash((layers . ("F" . #0#)) (net . 8) (shape . "rect") (type . "thru_hole") (x . 0) (y . 0)))) (x . 107.5) (y . 95)) #hash((pads . (#hash((layers . ("F" . #0#)) (net . 1) (shape . "circle") (type . "thru_hole") (x . 7.5) (y . 0)) #hash((layers . ("F" . #0#)) (net . 8) (shape . "rect") (type . "thru_hole") (x . 0) (y . 0)))) (x . 112.5) (y . 111)) #hash((pads . (#hash((layers . ("F" . #0#)) (net . 56) (shape . "oval") (type . "thru_hole") (x . 0) (y . 2.54)) #hash((layers . ("F" . #0#)) (net . 55) (shape . "rect") (type . "thru_hole") (x . 0) (y . 0)))) (x . 200.7) (y . 121.6)) #hash((pads . (#hash((layers . ("F" . #0#)) (net . 77) (shape . "oval") (type . "thru_hole") (x . 0) (y . 12.7)) #hash((layers . ("F" . #0#)) (net . 11) (shape . "oval") (type . "thru_hole") (x . 0) (y . 10.16)) #hash((layers . ("F" . #0#)) (net . 57) (shape . "oval") (type . "thru_hole") (x . 0) (y . 7.62)) #hash((layers . ("F" . #0#)) (net . 1) (shape . "oval") (type . "thru_hole") (x . 0) (y . 5.08)) #hash((layers . ("F" . #0#)) (net . 58) (shape . "oval") (type . "thru_hole") (x . 0) (y . 2.54)) #hash((layers . ("F" . #0#)) (net . 2) (shape . "rect") (type . "thru_hole") (x . 0) (y . 0)))) (x . 210.9) (y . 117.8)) #hash((pads . (#hash((layers . ()) (net . 1) (shape . "rect") (type . "smd") (x . 1.525) (y . 0)) #hash((layers . ()) (net . 6) (shape . "rect") (type . "smd") (x . -0.85) (y . 0)))) (x . 187.652468) (y . 120.698646)) #hash((pads . (#hash((layers . ()) (net . 12) (shape . "rect") (type . "smd") (x . 0.8) (y . 0)) #hash((layers . ()) (net . 2) (shape . "rect") (type . "smd") (x . -0.8) (y . 0)))) (x . 188.1) (y . 96.8)) #hash((pads . (#hash((layers . ()) (net . 10) (shape . "rect") (type . "smd") (x . 0.88) (y . 0)) #hash((layers . ()) (net . 8) (shape . "rect") (type . "smd") (x . -0.88) (y . 0)))) (x . 194.1) (y . 122.9)) #hash((pads . (#hash((layers . ()) (net . 66) (shape . "rect") (type . "smd") (x . 0.8) (y . 0)) #hash((layers . ()) (net . 1) (shape . "rect") (type . "smd") (x . -0.8) (y . 0)))) (x . 204.1) (y . 122.4)) #hash((pads . (#hash((layers . ()) (net . 67) (shape . "rect") (type . "smd") (x . 0.8) (y . 0)) #hash((layers . ()) (net . 1) (shape . "rect") (type . "smd") (x . -0.8) (y . 0)))) (x . 204.1) (y . 128.2)) #hash((pads . ()) (x . 200.8) (y . 111.9)) #hash((pads . (#hash((layers . ("F" . #0#)) (net . ()) (shape . "circle") (type . "np_thru_hole") (x . 0) (y . 0)))) (x . 113.5) (y . 88)) #hash((pads . (#hash((layers . ("F" . #0#)) (net . ()) (shape . "circle") (type . "np_thru_hole") (x . 0) (y . 0)))) (x . 198) (y . 88)) #hash((pads . (#hash((layers . ("F" . #0#)) (net . ()) (shape . "circle") (type . "np_thru_hole") (x . 0) (y . 0)))) (x . 198) (y . 127.5)) #hash((pads . (#hash((layers . ()) (net . 2) (shape . "rect") (type . "smd") (x . -2.75) (y . -4.35)) #hash((layers . ()) (net . 1) (shape . "rect") (type . "smd") (x . -2.25) (y . -4.35)) #hash((layers . ()) (net . 61) (shape . "rect") (type . "smd") (x . -1.75) (y . -4.35)) #hash((layers . ()) (net . 60) (shape . "rect") (type . "smd") (x . -1.25) (y . -4.35)) #hash((layers . ()) (net . 13) (shape . "rect") (type . "smd") (x . -0.75) (y . -4.35)) #hash((layers . ()) (net . 22) (shape . "rect") (type . "smd") (x . -0.25) (y . -4.35)) #hash((layers . ()) (net . 21) (s

In [690]:
(define clauses (match sexp
       [`(kicad_pcb
          ,clauses ...)
         clauses]))

In [691]:
(define module-clauses (filter (lambda (x)
              (match x
                     [`(module ,_ ...) #t]
                     [else #f]))
        clauses))

In [692]:
(define modules
      (for/list ([c module-clauses]
          [i (in-naturals)])
                (display i) (display " ")
         (module-clause->module c)))

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 

In [694]:
(module-clause->module (fourth module-clauses))

'#hash((pads . ()) (x . 205.6) (y . 89.9))

# Load data

In [2]:
(define sexp (parse-kicad-pcb "./benchmarks/cantact.kicad_pcb"))

In [82]:
;; clauses
(define clauses (match sexp
       [`(kicad_pcb
          ,clauses ...)
         clauses]))

# Examine the kicad_pcb structure

In [85]:
;; examine the first clauses
(remove-duplicates (map first clauses))

'(version host general page layers setup net net_class module gr_text gr_line segment via zone)

In [301]:
;; pretty-print is very slow in jupyter notebook
(display (pretty-format sexp))

'(kicad_pcb
  (version 4)
  (host pcbnew "(2014-10-22 BZR 5216)-product")
  (general
   (links 86)
   (no_connects 0)
   (area 108.219478 90.125 163.651363 155.28002)
   (thickness 1.6)
   (drawings 17)
   (tracks 355)
   (zones 0)
   (modules 40)
   (nets 62))
  (page A4)
  (layers
   (0 F.Cu signal)
   (31 B.Cu signal)
   (32 B.Adhes user hide)
   (33 F.Adhes user hide)
   (34 B.Paste user hide)
   (35 F.Paste user hide)
   (36 B.SilkS user)
   (37 F.SilkS user)
   (38 B.Mask user hide)
   (39 F.Mask user hide)
   (40 Dwgs.User user hide)
   (41 Cmts.User user)
   (42 Eco1.User user)
   (43 Eco2.User user)
   (44 Edge.Cuts user)
   (45 Margin user)
   (46 B.CrtYd user)
   (47 F.CrtYd user)
   (48 B.Fab user)
   (49 F.Fab user))
  (setup
   (last_trace_width 0.2032)
   (trace_clearance 0.2032)
   (zone_clearance 0.508)
   (zone_45_only yes)
   (trace_min 0.2032)
   (segment_width 0.2)
   (edge_width 0.1)
   (via_size 0.889)
   (via_drill 0.635)
   (via_min_size 0.889)
   (via_min_dril

# Extract the data

In [ ]:
(define area (kicad->area sexp))
area

## Net

In [94]:
;; net clauses
(filter (lambda (x)
              (match x
                     [`(net ,_ ...) #t]
                     [else #f]))
        clauses)

'((net 0 "") (net 1 +3.3V) (net 2 GND) (net 3 +5V) (net 4 /USB_DM) (net 5 /USB_DP) (net 6 /SWDIO) (net 7 /SWCLK) (net 8 "Net-(P3-Pad8)") (net 9 /CAN_RX) (net 10 /CAN_TX) (net 11 "Net-(C5-Pad1)") (net 12 "Net-(C9-Pad1)") (net 13 "Net-(D1-Pad1)") (net 14 "Net-(D1-Pad2)") (net 15 "Net-(D2-Pad1)") (net 16 "Net-(D2-Pad2)") (net 17 "Net-(JP1-Pad2)") (net 18 /DB9_1) (net 19 /DB9_2) (net 20 /DB9_3) (net 21 "Net-(JP4-Pad1)") (net 22 "Net-(JP2-Pad1)") (net 23 "Net-(U1-Pad2)") (net 24 "Net-(U1-Pad3)") (net 25 "Net-(U1-Pad4)") (net 26 "Net-(JP2-Pad2)") (net 27 "Net-(U3-Pad5)") (net 28 /DB9_5) (net 29 /NRST) (net 30 "Net-(U1-Pad10)") (net 31 "Net-(U1-Pad11)") (net 32 "Net-(U1-Pad12)") (net 33 "Net-(U1-Pad13)") (net 34 "Net-(U1-Pad14)") (net 35 "Net-(U1-Pad15)") (net 36 "Net-(U1-Pad16)") (net 37 "Net-(U1-Pad17)") (net 38 "Net-(U1-Pad20)") (net 39 "Net-(U1-Pad21)") (net 40 "Net-(U1-Pad22)") (net 41 "Net-(U1-Pad25)") (net 42 "Net-(U1-Pad26)") (net 43 "Net-(U1-Pad27)") (net 44 "Net-(U1-Pad28)") (net 45 "Net-(U1-Pad29)") (net 46 "Net-(U1-Pad30)") (net 47 "Net-(U1-Pad31)") (net 48 "Net-(U1-Pad38)") (net 49 "Net-(U1-Pad39)") (net 50 "Net-(U1-Pad40)") (net 51 "Net-(U1-Pad41)") (net 52 "Net-(U1-Pad42)") (net 53 "Net-(U1-Pad43)") (net 54 "Net-(P1-Pad5)") (net 55 "Net-(P1-Pad6)") (net 56 "Net-(P3-Pad4)") (net 57 "Net-(P3-Pad9)") (net 58 "Net-(P3-Pad6)") (net 59 "Net-(JP2-Pad3)") (net 60 "Net-(JP2-Pad4)") (net 61 /DB9_7))

## Module

In [96]:
;; module clauses
;; net clauses
(define module-clauses (filter (lambda (x)
              (match x
                     [`(module ,_ ...) #t]
                     [else #f]))
        clauses))

In [98]:
(display (pretty-format (first module-clauses)))

'(module Connect:DB9MC (layer F.Cu)
   (tedit 54DA4B60)
   (tstamp 54735AF4)
   (at 142.7 98.9)
   (descr "Connecteur DB9 male couche")
   (tags "CONN DB9")
   (path /54725006)
   (fp_text
    reference
    P3
    (at -17.2 -7.35 180)
    (layer F.SilkS)
    (effects (font (size 1 1) (thickness 0.15))))
   (fp_text
    value
    DB9
    (at 1.27 -3.81)
    (layer F.SilkS)
    hide
    (effects (font (thickness 0.3048))))
   (fp_line
    (start 16.4084 2.286)
    (end 16.4084 -8.0264)
    (layer F.SilkS)
    (width 0.15))
   (fp_line
    (start -16.1544 2.2606)
    (end -16.1544 -8)
    (layer F.SilkS)
    (width 0.15))
   (fp_line
    (start -16.129 -8.0264)
    (end 16.37 -8.0264)
    (layer F.SilkS)
    (width 0.15))
   (fp_line
    (start -16.129 2.286)
    (end 16.383 2.286)
    (layer F.SilkS)
    (width 0.15))
   (pad
    ""
    thru_hole
    circle
    (at 12.827 -1.27)
    (size 3.81 3.81)
    (drill 3.048)
    (layers *.Cu *.Mask F.SilkS))
   (pad
    ""
    thru_hole
    circ

In [153]:
(struct Module
        (x y pads)
        #:prefab)

In [154]:
(Module 1 2 3)

'#s(Module 1 2 3)

In [529]:
(struct Pad
        (type shape x y net layers)
        #:prefab)

In [525]:
(display (pretty-format (list-ref module-clauses 2)))

'(module Connect:USB_B (layer F.Cu)
   (tedit 54B5C750)
   (tstamp 5473662E)
   (at 133.3 141.4)
   (tags USB)
   (path /546FAFF9)
   (fp_text
    reference
    P1
    (at -7.15 9.625)
    (layer F.SilkS)
    (effects (font (size 1 1) (thickness 0.15))))
   (fp_text
    value
    USB
    (at 0 0)
    (layer F.SilkS)
    hide
    (effects (font (thickness 0.3048))))
   (fp_line
    (start -6.096 10.287)
    (end 6.096 10.287)
    (layer F.SilkS)
    (width 0.15))
   (fp_line
    (start 6.096 10.287)
    (end 6.096 -6.731)
    (layer F.SilkS)
    (width 0.15))
   (fp_line
    (start 6.096 -6.731)
    (end -6.096 -6.731)
    (layer F.SilkS)
    (width 0.15))
   (fp_line
    (start -6.096 -6.731)
    (end -6.096 10.287)
    (layer F.SilkS)
    (width 0.15))
   (pad
    1
    thru_hole
    circle
    (at 1.27 -4.699)
    (size 1.524 1.524)
    (drill 0.8128)
    (layers *.Cu *.Mask F.SilkS)
    (net 3 +5V))
   (pad
    2
    thru_hole
    circle
    (at -1.27 -4.699)
    (size 1.524 1.524)


In [627]:
(module-clause->module (list-ref module-clauses 33))

'#hash((pads . ()) (x . 133.2) (y . 147))

In [628]:
(module-clause->module (first module-clauses))

'#hash((pads . (#hash((layers . ("F" . #0=("B"))) (net . ()) (shape . "circle") (type . "thru_hole") (x . 12.827) (y . -1.27)) #hash((layers . ("F" . #0#)) (net . ()) (shape . "circle") (type . "thru_hole") (x . -12.573) (y . -1.27)) #hash((layers . ("F" . #0#)) (net . 18) (shape . "rect") (type . "thru_hole") (x . 5.588) (y . 1.27)) #hash((layers . ("F" . #0#)) (net . 19) (shape . "circle") (type . "thru_hole") (x . 2.794) (y . 1.27)) #hash((layers . ("F" . #0#)) (net . 20) (shape . "circle") (type . "thru_hole") (x . 0) (y . 1.27)) #hash((layers . ("F" . #0#)) (net . 56) (shape . "circle") (type . "thru_hole") (x . -2.667) (y . 1.27)) #hash((layers . ("F" . #0#)) (net . 28) (shape . "circle") (type . "thru_hole") (x . -5.461) (y . 1.27)) #hash((layers . ("F" . #0#)) (net . 57) (shape . "circle") (type . "thru_hole") (x . -4.064) (y . -1.27)) #hash((layers . ("F" . #0#)) (net . 8) (shape . "circle") (type . "thru_hole") (x . -1.27) (y . -1.27)) #hash((layers . ("F" . #0#)) (net . 61) (shape . "circle") (type . "thru_hole") (x . 1.397) (y . -1.27)) #hash((layers . ("F" . #0#)) (net . 58) (shape . "circle") (type . "thru_hole") (x . 4.191) (y . -1.27)))) (x . 142.7) (y . 98.9))

In [629]:
(define modules
  (for/list ([c module-clauses]
      [i (in-naturals)])
;;      (display i) (display " ")
     (module-clause->module c)))
modules

'(#hash((pads . (#hash((layers . ("F" . #0=("B"))) (net . ()) (shape . "circle") (type . "thru_hole") (x . 12.827) (y . -1.27)) #hash((layers . ("F" . #0#)) (net . ()) (shape . "circle") (type . "thru_hole") (x . -12.573) (y . -1.27)) #hash((layers . ("F" . #0#)) (net . 18) (shape . "rect") (type . "thru_hole") (x . 5.588) (y . 1.27)) #hash((layers . ("F" . #0#)) (net . 19) (shape . "circle") (type . "thru_hole") (x . 2.794) (y . 1.27)) #hash((layers . ("F" . #0#)) (net . 20) (shape . "circle") (type . "thru_hole") (x . 0) (y . 1.27)) #hash((layers . ("F" . #0#)) (net . 56) (shape . "circle") (type . "thru_hole") (x . -2.667) (y . 1.27)) #hash((layers . ("F" . #0#)) (net . 28) (shape . "circle") (type . "thru_hole") (x . -5.461) (y . 1.27)) #hash((layers . ("F" . #0#)) (net . 57) (shape . "circle") (type . "thru_hole") (x . -4.064) (y . -1.27)) #hash((layers . ("F" . #0#)) (net . 8) (shape . "circle") (type . "thru_hole") (x . -1.27) (y . -1.27)) #hash((layers . ("F" . #0#)) (net . 61) (shape . "circle") (type . "thru_hole") (x . 1.397) (y . -1.27)) #hash((layers . ("F" . #0#)) (net . 58) (shape . "circle") (type . "thru_hole") (x . 4.191) (y . -1.27)))) (x . 142.7) (y . 98.9)) #hash((pads . (#hash((layers . ("F" . #0#)) (net . 1) (shape . "rect") (type . "thru_hole") (x . -1.27) (y . 0)) #hash((layers . ("F" . #0#)) (net . 17) (shape . "oval") (type . "thru_hole") (x . 1.27) (y . 0)))) (x . 158) (y . 135)) #hash((pads . (#hash((layers . ("F" . #0#)) (net . 3) (shape . "circle") (type . "thru_hole") (x . 1.27) (y . -4.699)) #hash((layers . ("F" . #0#)) (net . 4) (shape . "circle") (type . "thru_hole") (x . -1.27) (y . -4.699)) #hash((layers . ("F" . #0#)) (net . 5) (shape . "circle") (type . "thru_hole") (x . -1.27) (y . -2.70002)) #hash((layers . ("F" . #0#)) (net . 2) (shape . "circle") (type . "thru_hole") (x . 1.27) (y . -2.70002)) #hash((layers . ("F" . #0#)) (net . 54) (shape . "circle") (type . "np_thru_hole") (x . 5.99948) (y . 0)) #hash((layers . ("F" . #0#)) (net . 55) (shape . "circle") (type . "thru_hole") (x . -5.99948) (y . 0)))) (x . 133.3) (y . 141.4)) #hash((pads . (#hash((layers . ("F" . #0#)) (net . 1) (shape . "rect") (type . "thru_hole") (x . -5.08) (y . 0)) #hash((layers . ("F" . #0#)) (net . 7) (shape . "oval") (type . "thru_hole") (x . -2.54) (y . 0)) #hash((layers . ("F" . #0#)) (net . 2) (shape . "oval") (type . "thru_hole") (x . 0) (y . 0)) #hash((layers . ("F" . #0#)) (net . 6) (shape . "oval") (type . "thru_hole") (x . 2.54) (y . 0)) #hash((layers . ("F" . #0#)) (net . 29) (shape . "oval") (type . "thru_hole") (x . 5.08) (y . 0)))) (x . 158) (y . 145)) #hash((pads . (#hash((layers . #1=("F")) (net . 17) (shape . "rect") (type . "smd") (x . -0.762) (y . 0)) #hash((layers . #1#) (net . 2) (shape . "rect") (type . "smd") (x . 0.762) (y . 0)))) (x . 153.2) (y . 137.1)) #hash((pads . (#hash((layers . ("F" . #0#)) (net . 3) (shape . "circle") (type . "thru_hole") (x . 0) (y . 0)))) (x . 143.9845) (y . 149.409)) #hash((pads . (#hash((layers . ("F" . #0#)) (net . 9) (shape . "circle") (type . "thru_hole") (x . 0) (y . 0)))) (x . 146.7785) (y . 128.1365)) #hash((pads . (#hash((layers . ("F" . #0#)) (net . 10) (shape . "circle") (type . "thru_hole") (x . 0) (y . 0)))) (x . 146.334) (y . 124.4535)) #hash((pads . (#hash((layers . #1#) (net . 1) (shape . "rect") (type . "smd") (x . -4.35) (y . -2.75)) #hash((layers . #1#) (net . 23) (shape . "rect") (type . "smd") (x . -4.35) (y . -2.25)) #hash((layers . #1#) (net . 24) (shape . "rect") (type . "smd") (x . -4.35) (y . -1.75)) #hash((layers . #1#) (net . 25) (shape . "rect") (type . "smd") (x . -4.35) (y . -1.25)) #hash((layers . #1#) (net . 11) (shape . "rect") (type . "smd") (x . -4.35) (y . -0.75)) #hash((layers . #1#) (net . 12) (shape . "rect") (type . "smd") (x . -4.35) (y . -0.25)) #hash((layers . #1#) (net . 29) (shape . "rect") (type . "smd") (x . -4.35) (y . 0.25)) #hash((layers . #1#) (net . 2) (shape . "rect") (type . "smd") (x . -4.35) (y . 0.75)) #

# Output area and modules to a json object, and parse in Python

In [630]:
area

'(108.219478 90.125 163.651363 155.28002)

In [631]:
(display (pretty-format (first modules)))

'#hash((pads
        .
        (#hash((layers . ("F" "B"))
               (net . ())
               (shape . "circle")
               (type . "thru_hole")
               (x . 12.827)
               (y . -1.27))
         #hash((layers . ("F" "B"))
               (net . ())
               (shape . "circle")
               (type . "thru_hole")
               (x . -12.573)
               (y . -1.27))
         #hash((layers . ("F" "B"))
               (net . 18)
               (shape . "rect")
               (type . "thru_hole")
               (x . 5.588)
               (y . 1.27))
         #hash((layers . ("F" "B"))
               (net . 19)
               (shape . "circle")
               (type . "thru_hole")
               (x . 2.794)
               (y . 1.27))
         #hash((layers . ("F" "B"))
               (net . 20)
               (shape . "circle")
               (type . "thru_hole")
               (x . 0)
               (y . 1.27))
         #hash((layers . ("F" "B"))
            

In [634]:
(jsexpr? modules)

#t

In [637]:
(display (pretty-format (jsexpr->string (first modules))))

"{\"pads\":[{\"layers\":[\"F\",\"B\"],\"net\":[],\"shape\":\"circle\",\"type\":\"thru_hole\",\"x\":12.827,\"y\":-1.27},{\"layers\":[\"F\",\"B\"],\"net\":[],\"shape\":\"circle\",\"type\":\"thru_hole\",\"x\":-12.573,\"y\":-1.27},{\"layers\":[\"F\",\"B\"],\"net\":18,\"shape\":\"rect\",\"type\":\"thru_hole\",\"x\":5.588,\"y\":1.27},{\"layers\":[\"F\",\"B\"],\"net\":19,\"shape\":\"circle\",\"type\":\"thru_hole\",\"x\":2.794,\"y\":1.27},{\"layers\":[\"F\",\"B\"],\"net\":20,\"shape\":\"circle\",\"type\":\"thru_hole\",\"x\":0,\"y\":1.27},{\"layers\":[\"F\",\"B\"],\"net\":56,\"shape\":\"circle\",\"type\":\"thru_hole\",\"x\":-2.667,\"y\":1.27},{\"layers\":[\"F\",\"B\"],\"net\":28,\"shape\":\"circle\",\"type\":\"thru_hole\",\"x\":-5.461,\"y\":1.27},{\"layers\":[\"F\",\"B\"],\"net\":57,\"shape\":\"circle\",\"type\":\"thru_hole\",\"x\":-4.064,\"y\":-1.27},{\"layers\":[\"F\",\"B\"],\"net\":8,\"shape\":\"circle\",\"type\":\"thru_hole\",\"x\":-1.27,\"y\":-1.27},{\"layers\":[\"F\",\"B\"],\"net\":61,\"s

In [642]:
(call-with-output-file "out.json"
                       #:exists 'replace
                     (lambda (out)
                       (write-json (hash 'modules modules
                                         'area area)
                                   out)))

# Output input matrix for youbiao

https://github.com/ybiao-he/circuit_routing/blob/master/Input_format.md

The input PCB board is a matrix whose elements has the following meanings:

1. the area that the path can pass has value 0;

2. the area of obstacles that the path can not pass has value 1, for example, the body of a chip;

3. the net to be connected is represented by a value larger than 1, pins of the same net are represented by the same number. For example, pins of net 1 have a value of 2.

```
0000000
0200000
0002110
0000110
0000000
```

Note that now we can only deal with simple circuit boards with the following limits:

1. circuit only contains one layer, that is we do not need to consider via

2. the total number of nets is smaller than 10, maybe it also works if the circuit has more than 10 nets, but it is best not to exceed 10. If we only have circuits with more than 10 nets, then keep the one with the least nets, and we can try.

3. each net contains two pins.

4. current board size (input matrix size) is 30*30.

In [ ]:
;; 1. get the box of board
;; 2. assign a grid size
;; 2. get all modules
;; 2. put modules onto the board
;; 3. put pins onto the board

In [310]:
area

'(108.219478 90.125 163.651363 155.28002)

In [345]:
(first modules)

'#s(Module 142.7 98.9 (#s(Pad thru_hole circle 12.827 -1.27 ()) #s(Pad thru_hole circle -12.573 -1.27 ()) #s(Pad thru_hole rect 5.588 1.27 18) #s(Pad thru_hole circle 2.794 1.27 19) #s(Pad thru_hole circle 0 1.27 20) #s(Pad thru_hole circle -2.667 1.27 56) #s(Pad thru_hole circle -5.461 1.27 28) #s(Pad thru_hole circle -4.064 -1.27 57) #s(Pad thru_hole circle -1.27 -1.27 8) #s(Pad thru_hole circle 1.397 -1.27 61) #s(Pad thru_hole circle 4.191 -1.27 58)))

In [490]:
(define (make-shift x1 y1 grid)
  (lambda (x y)
      (values (exact-round (/ (- x x1) grid))
              (exact-round (/ (- y y1) grid)))))

In [508]:
(define (add-to-matrix matrix module shift)
  ;; 1. add all pads
  ;; 2. TODO add bounding box
  ;; 3. relate nets
  (for ([pad (Module-pads module)]
        #:when (not (empty? (Pad-net pad))))
       ;; compute the coordinate
       (let-values ([(x y) (shift (+ (Pad-x pad) (Module-x module))
                                  (+ (Pad-y pad) (Module-y module)))])
;;                    (display (~a "setting " x "," y))
                   (vector-set! (vector-ref matrix x) y (Pad-net pad))))
  )

In [522]:
(define (vis-board m)
  ;; convert all entries into format strings
  (display (string-replace 
            (parameterize ([pretty-print-columns 700])
                          (pretty-format
                            (for/list ([mi m])
                              (for/list ([i mi])
                                          (~r i #:min-width 2)
                                        )))) "\"" "")))

In [ ]:
(define (twoify-nets mat)
  ;; make all nets 2-pad-net
  ;; 1. turn mat into 1-d array
  ;; 2. turn number into 1-1, 1-2, 1-3, etc
  ;; 3. turn string into numbers
  (define h (make-hash))
  (for ([row mat])
       (for ([i (range (length row))])
            (define v (vector-ref row i))
            (if (hash-has h v)
                ())
            (vector-set! row i (~a v ))
  )

In [523]:
(define (board->matrix area modules)
  ;; 1. create the matrix
  (define SIZE 100)
  (match-define (list x1 y1 x2 y2) area)
  (define grid (/ (- x2 x1) SIZE))
  (define shift (make-shift x1 y1 grid))
  ;; create the matrix
  (define nx (+ SIZE 1))
  (define ny (+ (exact-round (/ (- y2 y1) grid)) 1))
;;   (displayln (~a "nx:" nx ", ny:" ny))
  (define mat (for/vector ([i (in-range nx)])
                        (for/vector ([j (in-range ny)])
                                    0)))
  ;; insert
  (for ([module modules])
       (add-to-matrix mat module shift))
  mat
  )

In [524]:
(vis-board (board->matrix area modules))

'(( 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0)
  ( 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0)
  ( 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0

In [373]:
(vector-set! (vector-ref matrix 1) 2 3)

In [378]:
(add-to-matrix matrix (first modules))

12,311,311,310,39,39,210,211,212,2

In [319]:
modules

'(#s(Module 142.7 98.9 (#s(Pad thru_hole circle 12.827 -1.27 ()) #s(Pad thru_hole circle -12.573 -1.27 ()) #s(Pad thru_hole rect 5.588 1.27 18) #s(Pad thru_hole circle 2.794 1.27 19) #s(Pad thru_hole circle 0 1.27 20) #s(Pad thru_hole circle -2.667 1.27 56) #s(Pad thru_hole circle -5.461 1.27 28) #s(Pad thru_hole circle -4.064 -1.27 57) #s(Pad thru_hole circle -1.27 -1.27 8) #s(Pad thru_hole circle 1.397 -1.27 61) #s(Pad thru_hole circle 4.191 -1.27 58))) #s(Module 158 135 (#s(Pad thru_hole rect -1.27 0 1) #s(Pad thru_hole oval 1.27 0 17))) #s(Module 133.3 141.4 (#s(Pad thru_hole circle 1.27 -4.699 3) #s(Pad thru_hole circle -1.27 -4.699 4) #s(Pad thru_hole circle -1.27 -2.70002 5) #s(Pad thru_hole circle 1.27 -2.70002 2) #s(Pad np_thru_hole circle 5.99948 0 54) #s(Pad thru_hole circle -5.99948 0 55))) #s(Module 158 145 (#s(Pad thru_hole rect -5.08 0 1) #s(Pad thru_hole oval -2.54 0 7) #s(Pad thru_hole oval 0 0 2) #s(Pad thru_hole oval 2.54 0 6) #s(Pad thru_hole oval 5.08 0 29))) #s(Module 153.2 137.1 (#s(Pad smd rect -0.762 0 17) #s(Pad smd rect 0.762 0 2))) #s(Module 143.9845 149.409 (#s(Pad thru_hole circle 0 0 3))) #s(Module 146.7785 128.1365 (#s(Pad thru_hole circle 0 0 9))) #s(Module 146.334 124.4535 (#s(Pad thru_hole circle 0 0 10))) #s(Module 136.1105 125.4695 (#s(Pad smd rect -4.35 -2.75 1) #s(Pad smd rect -4.35 -2.25 23) #s(Pad smd rect -4.35 -1.75 24) #s(Pad smd rect -4.35 -1.25 25) #s(Pad smd rect -4.35 -0.75 11) #s(Pad smd rect -4.35 -0.25 12) #s(Pad smd rect -4.35 0.25 29) #s(Pad smd rect -4.35 0.75 2) #s(Pad smd rect -4.35 1.25 1) #s(Pad smd rect -4.35 1.75 30) #s(Pad smd rect -4.35 2.25 31) #s(Pad smd rect -4.35 2.75 32) #s(Pad smd rect -2.75 4.35 33) #s(Pad smd rect -2.25 4.35 34) #s(Pad smd rect -1.75 4.35 35) #s(Pad smd rect -1.25 4.35 36) #s(Pad smd rect -0.75 4.35 37) #s(Pad smd rect -0.25 4.35 13) #s(Pad smd rect 0.25 4.35 15) #s(Pad smd rect 0.75 4.35 38) #s(Pad smd rect 1.25 4.35 39) #s(Pad smd rect 1.75 4.35 40) #s(Pad smd rect 2.25 4.35 2) #s(Pad smd rect 2.75 4.35 1) #s(Pad smd rect 4.35 2.75 41) #s(Pad smd rect 4.35 2.25 42) #s(Pad smd rect 4.35 1.75 43) #s(Pad smd rect 4.35 1.25 44) #s(Pad smd rect 4.35 0.75 45) #s(Pad smd rect 4.35 0.25 46) #s(Pad smd rect 4.35 -0.25 47) #s(Pad smd rect 4.35 -0.75 4) #s(Pad smd rect 4.35 -1.25 5) #s(Pad smd rect 4.35 -1.75 6) #s(Pad smd rect 4.35 -2.25 2) #s(Pad smd rect 4.35 -2.75 1) #s(Pad smd rect 2.75 -4.35 7) #s(Pad smd rect 2.25 -4.35 48) #s(Pad smd rect 1.75 -4.35 49) #s(Pad smd rect 1.25 -4.35 50) #s(Pad smd rect 0.75 -4.35 51) #s(Pad smd rect 0.25 -4.35 52) #s(Pad smd rect -0.25 -4.35 53) #s(Pad smd rect -0.75 -4.35 17) #s(Pad smd rect -1.25 -4.35 9) #s(Pad smd rect -1.75 -4.35 10) #s(Pad smd rect -2.25 -4.35 2) #s(Pad smd rect -2.75 -4.35 1))) #s(Module 145.5085 146.488 (#s(Pad smd rect -0.9525 1.27 3) #s(Pad smd rect 0.9525 1.27 3) #s(Pad smd rect -0.9525 -1.27 1) #s(Pad smd rect 0 1.27 2) #s(Pad smd rect 0.9525 -1.27 ()))) #s(Module 152.1125 126.295 (#s(Pad smd rect -1.905 -3.175 2) #s(Pad smd rect -0.635 -3.175 26) #s(Pad smd rect 0.635 -3.175 21) #s(Pad smd rect 1.905 -3.175 27) #s(Pad smd rect 1.905 3.175 9) #s(Pad smd rect 0.635 3.175 3) #s(Pad smd rect -0.635 3.175 2) #s(Pad smd rect -1.905 3.175 10))) #s(Module 148.62 116.389 (#s(Pad smd rect -4.84886 0 11) #s(Pad smd rect 4.84886 0 12))) #s(Module 142.905 121.85 (#s(Pad smd rect -0.75 0 1) #s(Pad smd rect 0.75 0 2))) #s(Module 144.683 121.85 (#s(Pad smd rect -0.75 0 1) #s(Pad smd rect 0.75 0 2))) #s(Module 146.461 121.85 (#s(Pad smd rect -0.75 0 1) #s(Pad smd rect 0.75 0 2))) #s(Module 131.5385 132.4545 (#s(Pad smd rect -0.75 0 1) #s(Pad smd rect 0.75 0 2))) #s(Module 150.2 121.2 (#s(Pad smd rect -0.75 0 11) #s(Pad smd rect 0.75 0 2))) #s(Module 129.697 132.4545 (#s(Pad smd rect -0.75 0 1) #s(Pad smd rect 0.75 0 2))) #s(Module 127.919 132.4545 (#s(Pad smd rect -0.75 0 1) #s(Pad smd rect 0.75 0 2))) #s(Module 134.9675 118.04 (#s(Pad smd rect -0.75 0 1) #s(Pad smd rect 0.75 0 2))) #s(Module 154.8 

In [376]:
(define matrix (for/vector ([i (in-range SIZE)])
                  (for/vector ([j (in-range SIZE)])
                        0)))

In [482]:
(vis-board matrix)

'(( 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0)
  ( 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0)
  ( 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0)
  ( 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0)
  ( 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0)
  ( 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0)
  ( 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0)
  ( 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0)
  ( 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0)
  ( 0  0 57 28  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0)
  ( 0  0  8 56  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0)
  ( 0  0 61 20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0)
  ( 0  0 58 18  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0)
  ( 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0)
  ( 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0)
  ( 0  0  0  0  0  0  0  0  0  0  0  0  

In [379]:
(display (pretty-format matrix))

'#(#(0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0)
   #(0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0)
   #(0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0)
   #(0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0)
   #(0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0)
   #(0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0)
   #(0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0)
   #(0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0)
   #(0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0)
   #(0 0 57 28 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0)
   #(0 0 8 56 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0)
   #(0 0 61 20 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0)
   #(0 0 58 18 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0)
   #(0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0)
   #(0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0)
   #(0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0)
   #(0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0)
   #(0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0)
   #(0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0)
   #(0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0))

## Other

In [80]:
;; get nets
(match sexp
       [`(kicad_pcb
          ,clauses ...)
         (filter-not null?
                     (for/list ([clause clauses])
                      (match clause
                             [`(net ,netId ,netName) `(NET ,netId ,netName)]
                             [else null])))])

'((NET 0 "") (NET 1 +3.3V) (NET 2 GND) (NET 3 +5V) (NET 4 /USB_DM) (NET 5 /USB_DP) (NET 6 /SWDIO) (NET 7 /SWCLK) (NET 8 "Net-(P3-Pad8)") (NET 9 /CAN_RX) (NET 10 /CAN_TX) (NET 11 "Net-(C5-Pad1)") (NET 12 "Net-(C9-Pad1)") (NET 13 "Net-(D1-Pad1)") (NET 14 "Net-(D1-Pad2)") (NET 15 "Net-(D2-Pad1)") (NET 16 "Net-(D2-Pad2)") (NET 17 "Net-(JP1-Pad2)") (NET 18 /DB9_1) (NET 19 /DB9_2) (NET 20 /DB9_3) (NET 21 "Net-(JP4-Pad1)") (NET 22 "Net-(JP2-Pad1)") (NET 23 "Net-(U1-Pad2)") (NET 24 "Net-(U1-Pad3)") (NET 25 "Net-(U1-Pad4)") (NET 26 "Net-(JP2-Pad2)") (NET 27 "Net-(U3-Pad5)") (NET 28 /DB9_5) (NET 29 /NRST) (NET 30 "Net-(U1-Pad10)") (NET 31 "Net-(U1-Pad11)") (NET 32 "Net-(U1-Pad12)") (NET 33 "Net-(U1-Pad13)") (NET 34 "Net-(U1-Pad14)") (NET 35 "Net-(U1-Pad15)") (NET 36 "Net-(U1-Pad16)") (NET 37 "Net-(U1-Pad17)") (NET 38 "Net-(U1-Pad20)") (NET 39 "Net-(U1-Pad21)") (NET 40 "Net-(U1-Pad22)") (NET 41 "Net-(U1-Pad25)") (NET 42 "Net-(U1-Pad26)") (NET 43 "Net-(U1-Pad27)") (NET 44 "Net-(U1-Pad28)") (NET 45 "Net-(U1-Pad29)") (NET 46 "Net-(U1-Pad30)") (NET 47 "Net-(U1-Pad31)") (NET 48 "Net-(U1-Pad38)") (NET 49 "Net-(U1-Pad39)") (NET 50 "Net-(U1-Pad40)") (NET 51 "Net-(U1-Pad41)") (NET 52 "Net-(U1-Pad42)") (NET 53 "Net-(U1-Pad43)") (NET 54 "Net-(P1-Pad5)") (NET 55 "Net-(P1-Pad6)") (NET 56 "Net-(P3-Pad4)") (NET 57 "Net-(P3-Pad9)") (NET 58 "Net-(P3-Pad6)") (NET 59 "Net-(JP2-Pad3)") (NET 60 "Net-(JP2-Pad4)") (NET 61 /DB9_7))

In [61]:
;; match sexp
(match sexp
       [`(kicad_pcb 
          (version ,v)
          ,_ ...
          (general (links ,n) ,_ ...)
          ,_ ...
          ;; CAUTION I have to list the previous clause (setup ...) here,
          ;; otherwise (net )... willl get empty list
          (setup ,_ ...)
          (net ,netId ,netName) ...
          ;; FIXME what if the order of the clauses changes?
          (net_class ,_ ...) ...
          (module ,mName ,_ ...) ...
          ,_ ...) (values netId mName)])

'(0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61)

'(Connect:DB9MC Pin_Headers:Pin_Header_Straight_1x02 Connect:USB_B Pin_Headers:Pin_Header_Straight_1x05 SMD_Packages:SMD-0603 Connect:PINTST Connect:PINTST Connect:PINTST Housings_QFP:LQFP-48_7x7mm_Pitch0.5mm SMD_Packages:SOT-23-5 SMD_Packages:SOIC-8-N Crystals:Crystal_HC49-SD_SMD Capacitors_SMD:C_0603 Capacitors_SMD:C_0603 Capacitors_SMD:C_0603 Capacitors_SMD:C_0603 Capacitors_SMD:C_0603 Capacitors_SMD:C_0603 Capacitors_SMD:C_0603 Capacitors_SMD:C_0603 Capacitors_SMD:C_0603 Capacitors_SMD:C_0603 Capacitors_SMD:C_0603 Capacitors_SMD:C_0603 Capacitors_SMD:C_0603 Capacitors_SMD:C_0603 Capacitors_SMD:C_0603 Capacitors_SMD:C_0603 SMD_Packages:SMD-0603 SMD_Packages:SMD-0603 SMD_Packages:SMD-0603 SMD_Packages:SMD-0603 SMD_Packages:SMD-0603 Symbols:Symbol_OSHW-Logo_SilkScreen cantact:0015912040 cantact:0015912040 cantact:0015912040 cantact:0015912040 Mounting_Holes:MountingHole_2-5mm Mounting_Holes:MountingHole_2-5mm)